In [9]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git 
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

In [1]:
from datasets import load_dataset

data_ljp_criminal = load_dataset("lbox/lbox_open", "ljp_criminal")
data_ljp_criminal

Found cached dataset lbox_open (/home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae)


  0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'casetype', 'casename', 'facts', 'reason', 'label', 'ruling'],
        num_rows: 8400
    })
    validation: Dataset({
        features: ['id', 'casetype', 'casename', 'facts', 'reason', 'label', 'ruling'],
        num_rows: 1050
    })
    test: Dataset({
        features: ['id', 'casetype', 'casename', 'facts', 'reason', 'label', 'ruling'],
        num_rows: 1050
    })
    test2: Dataset({
        features: ['id', 'casetype', 'casename', 'facts', 'reason', 'label', 'ruling'],
        num_rows: 928
    })
})

In [2]:
data_ljp_criminal['train'][0]

{'id': 0,
 'casetype': 'criminal',
 'casename': '강제추행',
 'facts': '피고인은 2020. 9. 25. 23:45경 포항시 남구 B에 있는 피해자 C(여, 62세) 운영의 D에서 피해자와 함께 술을 마신 후 결제를 하고 나왔고, 피해자가 피고인을 배웅하자 갑자기 피해자의 양 손목을 잡고 포항시 남구 E에 있는 F 출입구까지 약 20m 가량 끌고 간 뒤, 피해자를 강제로 끌어안고 입을 맞추며 손으로 피해자의 가슴을 수회 만졌다.\n이로써 피고인은 피해자를 강제로 추행하였다.',
 'reason': '양형의 이유\n1. 법률상 처단형의 범위: 징역 1월 ∼ 10년\n2. 양형기준에 따른 권고형의 범위\n[유형의 결정] 성범죄 > 01. 일반적 기준 > 나. 강제추행죄(13세 이상 대상) > [제1유형] 일반강제추행\n[특별양형인자] 감경요소 : 처벌불원\n[권고영역 및 권고형의 범위] 감경영역, 징역 1월∼1년\n3. 선고형의 결정 :\n추행의 정도가 가볍지 않은 점을 고려하여 징역형으로 처벌하되, 피고인이 이 사건 범행을 인정하고 있는 점, 피해자와 합의하여 피해자가 피고인의 처벌을 원하지 않는 점, 그 밖에 피고인의 나이, 성행, 환경, 범행의 동기와 경위, 범행 후의 정황 등 이 사건 변론에 나타난 여러 양형조건을 종합하여 주문과 같이 형을 정한다.\n신상정보 등록 및 제출의무\n판시 범죄사실에 관한 유죄판결이 확정되는 경우 피고인은 성폭력범죄의 처벌 등에 관한 특례법 제42조 제1항에 의하여 신상정보 등록대상자에 해당하게 되므로, 같은 법 제43조에 따라 관할기관에 신상정보를 제출할 의무가 있다.',
 'label': {'text': '징역 6월',
  'fine_lv': 0,
  'imprisonment_with_labor_lv': 2,
  'imprisonment_without_labor_lv': 0},
 'ruling': {'text': '피고인을 징역 6월에 처한다.\n다만, 이 판결 확정일부터 2년간

In [3]:
data_ljp_criminal = data_ljp_criminal.map(
    lambda x: {'text': f"### 명령어: 판결 이유를 설명하시오.\n\n###사건 종류: {x['casename']}, 법률사실: {x['facts']}\n\n### 답변: {x['reason']}<|endoftext|>" }
)

Loading cached processed dataset at /home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae/cache-570694fe48447d4a.arrow
Loading cached processed dataset at /home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae/cache-725325677843d9f1.arrow
Loading cached processed dataset at /home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae/cache-5d688aa9e651816f.arrow
Loading cached processed dataset at /home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae/cache-f121e38cd06fa4a0.arrow


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_ckpt = "beomi/llama-2-ko-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bits=True, # 4bit quantization
    bnb_4bit_use_double_quant=True, # 양자화 상수 공유
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForCausalLM.from_pretrained(model_ckpt, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [13]:
tokenizer.pad_token = tokenizer.eos_token

def encode(examples):
    inputs = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)
    labels = tokenizer([ruling['text'] for ruling in examples['ruling']], truncation=True, padding='max_length', max_length=512)
    inputs['labels'] = labels['input_ids']  # the labels are the input_ids of the target text
    return inputs

dataset = data_ljp_criminal.map(encode, batched=True)

Loading cached processed dataset at /home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae/cache-57d7ac9d7a6b8c36.arrow
Loading cached processed dataset at /home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae/cache-270d40bce883bb8d.arrow
Loading cached processed dataset at /home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae/cache-7f139faa2ba0e5d3.arrow
Loading cached processed dataset at /home/yunho/.cache/huggingface/datasets/lbox___lbox_open/ljp_criminal/0.2.0/3d5761e2d9292b674a2adabbe7f4bc200d1985908e6f657e7953c9bf247da7ae/cache-d88989a4963dd0ea.arrow


In [12]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [7]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [18]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=8, # attention dim
    lora_alpha=32, # LoRA 스케일링 파라미터
    target_modules=["q_proj", "v_proj"], # LoRA 적용할 모듈
    # target_modules=["query_key_value"] : 오류 발생(이유 모름)
    lora_dropout=0.05, # LoRA 드롭아웃
    bias="none",
    task_type="CAUSAL_LM",
)

peft_config.inference_mode = False
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,860,050,432 || trainable%: 0.06114100824149743


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 2


In [20]:
dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
dataset['train']

Dataset({
    features: ['id', 'casetype', 'casename', 'facts', 'reason', 'label', 'ruling', 'text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8400
})

In [23]:
import transformers

trainer = transformers.Trainer(
    model=model,
    train_dataset=dataset["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False 
trainer.train()

Step,Training Loss
10,4.269900
20,3.875100
30,3.810100
40,3.510100
50,3.311900


KeyboardInterrupt: 

In [24]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()